In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
# from torchvision import transforms

In [67]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(1, 16, 5, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7,stride=2,padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 128, 9, stride=2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, 9, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 7,stride=2,padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 5, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 5, stride=2, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [71]:
x = torch.rand(8, 1, 1, 110, 150)

In [72]:
ae = Autoencoder()

In [73]:
out = ae.encoder(x.reshape((x.shape[0],x.shape[1],x.shape[3],x.shape[4])))

In [74]:
out.reshape(out.shape[0],-1).shape

torch.Size([8, 512])

In [58]:
28*28

784